# BerryBox Berry Segmentation

## Run inferencing using a trained YOLOv8 model

This notebook is meant to be run on a Windows PC using a CPU

In [2]:
import paramiko
import time
from scp import SCPClient
import pandas as pd
from datetime import datetime
from ultralytics import YOLO
import cv2



In [3]:
def create_ssh_client(hostname, username, password):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(hostname, username=username, password=password)
    return ssh

# A function that checks the nikon camera for connection and sets configurations
def setup_nikon_camera(ssh, camera: str = "Nikon DSC D7500", sleeps = 2):
    # First check that the Emlid device is connected
    print("Checking Nikon Camera connection...")
    
    stdin, stdout, stderr = ssh.exec_command("gphoto2 --auto-detect")
    det = stdout.read().decode("utf-8")
    if not camera in str(det):
        raise RuntimeError(camera + " is not connected!")
    
    # Kill gphoto processes
    ssh.exec_command("pkill -f gphoto2")
    
    # Set configurations
    command = "gphoto2 --set-config iso=100"
    ssh.exec_command(command)
    time.sleep(sleeps) # These must be here 
    command = "gphoto2 --set-config whitebalance=7"
    ssh.exec_command(command)
    time.sleep(sleeps)
    command = "gphoto2 --set-config /main/capturesettings/f-number=7.1"
    ssh.exec_command(command)
    time.sleep(sleeps)
    command = "gphoto2 --set-config /main/capturesettings/shutterspeed=25"
    ssh.exec_command(command)

    print(camera + " is connected and configured!")



def transfer_image(ssh, remote_path, local_path):
    scp = SCPClient(ssh.get_transport())
    scp.get(remote_path, local_path)
    scp.close()



In [8]:
hostname = "169.254.111.10"
username = "cranpi2"
password = "usdacran1"
try:
    ssh = create_ssh_client(hostname, username, password)
except:
    print("Could not connect to pi")

Count not connect to pi


In [5]:
ssh.

In [68]:
setup_nikon_camera(ssh = ssh)

Checking Nikon Camera connection...
Nikon DSC D7500 is connected and configured!


In [69]:
remote_image_path = "/home/cranpi2/berrybox/captured_image.jpg"
trigger_command = f"gphoto2 --capture-image-and-download --filename {remote_image_path} --force-overwrite"
ssh.exec_command(command=trigger_command)
time.sleep(3)

In [73]:
# 4. Transfer the image from Raspberry Pi to local machine
# 2. Capture barcode and wait for Enter
barcode = input("Scan the barcode (or type 'exit' to quit): ")

input("Press Enter to capture the image...")

local_image_path = f"./captured_image_{barcode}_{datetime.now().strftime('%Y%m%d%H%M%S')}.jpg"
transfer_image(ssh, remote_image_path, local_image_path)

In [45]:
ssh.close()